In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, classification_report
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import numpy as np
from scipy.spatial import distance
import numpy as np

In [3]:
df = pd.read_csv('fetal_health.csv')
df.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [4]:
x = df.drop(columns=['fetal_health'])
y = df['fetal_health']

In [5]:
le = LabelEncoder()
y = le.fit_transform(y)

In [6]:
df['fetal_health'] = y

In [7]:
x = df.drop(columns=['fetal_health'])
y = df['fetal_health']

In [8]:
x_init_train, x_left, y_init_train, y_left = train_test_split(x,y,train_size=0.01415, random_state=42)

In [11]:
list_acc = [0]*79
for j in range(10):
    ac = []
    x_train, y_train = x_init_train, y_init_train
    unlabel, x_test, label, y_test = train_test_split(x_left, y_left, test_size=0.50)
    samples = [x_train.shape[0]]

    for i in range(78):
        xgb_model = XGBClassifier(objective="multi:softprob", num_class=3, colsample_bytree=0.5, learning_rate=0.05, max_depth=5, n_estimators=200)
        xgb_model.fit(x_train, y_train)
        y_probab = xgb_model.predict_proba(unlabel)

        x_test = pd.concat([x_test, unlabel])
        y_test = pd.concat([y_test, label])
        y_pred_proba = xgb_model.predict_proba(x_test)
        y_pred = np.argmax(y_pred_proba, axis=1)
        ac.append((accuracy_score(y_test, y_pred)) * 100)

        max_probs = np.max(y_probab, axis=1)
        second_max_probs = np.partition(y_probab, -2, axis=1)[:, -2]
        prob_diff = max_probs - second_max_probs
        top_uncertain_indices = np.argsort(prob_diff)[:20]

        selected_points = [top_uncertain_indices[0]]
        remaining_points = top_uncertain_indices[1:]

        while len(selected_points) < 5:
            sum_distances = []

            for p in remaining_points:
                total_distance = 0

                for sp in selected_points:
                    selected_point_1d = np.ravel(sp)
                    remaining_point_1d = np.ravel(p)
                    total_distance += distance.euclidean(selected_point_1d, remaining_point_1d)

                sum_distances.append(total_distance)

            farthest_point_index = np.argmax(sum_distances)
            farthest_point = remaining_points[farthest_point_index]
            selected_points.append(farthest_point)
            remaining_points = np.delete(remaining_points, farthest_point_index, axis=0)

        x_train = pd.concat([x_train, unlabel.iloc[selected_points]])
        y_train = pd.concat([y_train, label.iloc[selected_points]])
        unlabel = unlabel.drop(unlabel.index[selected_points])
        label = label.drop(label.index[selected_points])
        samples.append(x_train.shape[0])

    xgb_model2 = XGBClassifier(objective="multi:softprob", num_class=3, random_state=42)
    xgb_model2.fit(x_train, y_train)
    x_test = pd.concat([x_test, unlabel])
    y_test = pd.concat([y_test, label])
    y_pred_proba = xgb_model2.predict_proba(x_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    ac.append((accuracy_score(y_test, y_pred)) * 100)
    list_acc = [list_acc[k] + ac[k] for k in range(len(list_acc))]

result = [x / 10 for x in list_acc]

print("final accuracy: ", result[-1])

final accuracy:  99.43873748812742
